In [ ]:
# %%
# filename: optim_backtestingpy_sma_with_sl_tp.ipynb

# --- 1. Imports and Setup ---
import numpy as np
import pandas as pd
import pandas_ta as ta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import EURUSD

# --- 2. Load Sample Data ---
print("Loading built-in EURUSD sample data...")
data = EURUSD.copy()
print("Data loaded successfully.")

In [ ]:
# %%
# --- 3. The Parameterized Strategy with SL/TP ---

class SmaCrossSLTP(Strategy):
    # --- Parameters for Optimization ---
    fast_period = 10
    slow_period = 40

    # New parameters for Stop-Loss and Take-Profit (as percentages)
    # Note: We express them as decimals, e.g., 1.5% SL is 0.015
    sl_percent = 1.5
    tp_percent = 3.0

    def init(self):
        # Calculate indicators internally for optimization
        self.fast_sma = self.I(ta.sma, pd.Series(self.data.Close), length=self.fast_period)
        self.slow_sma = self.I(ta.sma, pd.Series(self.data.Close), length=self.slow_period)

    def next(self):
        # Define SL and TP prices based on the current close
        sl_price = self.data.Close[-1] * (1 - self.sl_percent / 100)
        tp_price = self.data.Close[-1] * (1 + self.tp_percent / 100)

        # Entry logic
        if crossover(self.fast_sma, self.slow_sma):
            if not self.position:
                # When buying, provide the sl and tp prices
                self.buy(sl=sl_price, tp=tp_price)

        # The exit signal from the crossover is still valid
        elif crossover(self.slow_sma, self.fast_sma):
            self.position.close()

In [ ]:
# %%
# --- 4. Running the Optimization (FINAL, DEFINITIVE VERSION) ---
print("\n--- Starting Grid Search Optimization with SL/TP ---")

bt = Backtest(data, SmaCrossSLTP, cash=10000, commission=.002)

# Helper function to generate a range of floats
def frange(start, stop, step):
    return np.arange(start, stop, step).tolist()

# CORRECTED: The optimize() method with return_heatmap=True returns a TUPLE
# containing (stats_series, heatmap_data). We unpack this tuple into two variables.
stats, heatmap = bt.optimize(
    fast_period=range(10, 31, 10),
    slow_period=range(40, 61, 10),
    sl_percent=frange(0.5, 2.1, 0.5),
    tp_percent=frange(2.0, 5.1, 1.0),

    maximize='SQN',
    constraint=lambda p: p.fast_period < p.slow_period,
    return_heatmap=True # This is crucial for getting the tuple back
)

print("\n--- Optimization Results ---")
print("Best stats found:")
# The 'stats' variable now correctly holds the results Series
print(stats)

print("\nBest parameters found:")
# The strategy object is an attribute of the stats Series
print(stats._strategy)

# The 'heatmap' variable now correctly holds the heatmap data for the next cell

In [ ]:
# %%
# --- 5. Visualizing the Best Run and Heatmaps ---
from backtesting.lib import plot_heatmaps

print("\n--- Plotting the Best Strategy Run ---")
# The .plot() method will automatically show the SL/TP levels on the chart
bt.plot()

print("\n--- Plotting Optimization Heatmap ---")
# This will generate a heatmap for each pair of optimized parameters
plot_heatmaps(heatmap, agg='mean')